# Using the EconModelClass

**Table of contents**<a id='toc0_'></a>    
- 1. [Baseline Usage](#toc1_)    
- 2. [Copy, save and load](#toc2_)    
- 3. [To and from dictionary](#toc3_)    
- 4. [Numba](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

`EconModel` is a small package for easily working with economic models in Python. It has four objectives:

1. Make it easy to write well-structed code.
1. Provide standard functionality for copying, saving and loading.
1. Provide an easy interface to call [numba](http://numba.pydata.org/) JIT compilled functions.
1. Provide an easy interface to call C++ functions (not relevant in this course).

This notebook provides a simple example for using the `EconModelClass`. See documentation in [EconModel/README.md](https://github.com/NumEconCopenhagen/EconModel#econmodel).

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb

from EconModel import EconModelClass, jit

## 1. <a id='toc1_'></a>[Baseline Usage](#toc0_)

The model is **required** to have the following three methods:

1. `.settings()`: Choose fundamental settings.
1. `.setup()`: Set free parameters.
1. `.allocate()`: Set compound parameters and allocate arrays.

When the model is initialized the call order is:

1. `.settings()`
2. `.setup()` 
3. `.allocate()`

Per default the namespaces `.par` (for parameters), `.sol` (for solution objects) and `.sim` (for simulation objects) are available.

The philosophy of the package is that all the content of the model is contained in these namespaces. 

After the model has been initialized, no additional entries should be added to the namespaces, and each entry should only change *value(s)* (or shape for arrays), but never *type* (or number of dimensions for arrays).

In [2]:
class MyModelClass(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
            
        pass # nothing chosen here
    
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        sol = self.sol
        
        par.X = np.linspace(0,10,par.N)
        sol.Y = np.zeros(par.N)
        par.Z = np.ones(par.N,dtype=int)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        sol = self.sol
        
        sol.Y[:] = par.X*(par.a+par.b)*par.Z


In `.settings()` the user can change the default behavior by specifying: 

1. `self.savefolder = str`: Filepath to save in and load from (default: 'saved').
1. `self.namespaces = [str]`: List of namespaces (in addition to `.par`, `.sol`, `.sol`).
1. `self.other_attrs = [str]`: List of additional attributes to be copied and saved.


In [3]:
model = MyModelClass(name='example')

In [4]:
model.solve()

**Print description:**

In [5]:
print(model)

Modelclass: MyModelClass
Name: example

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 Z = ndarray[int32] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sim:
 memory: 0.0 mb



**Updating at initialization.** Called after `.setup()`, but before `.allocate()`.

In [6]:
model_alt = MyModelClass(name='alt',par={'a':3.0,'b':1.2})
print(model_alt.par.a)

3.0


**Unpack for later use:**

In [7]:
par = model.par
sol = model.sol

**Under the hood:** Each namespace is techically a `SimpleNamespace` (from `types`)

In [8]:
type(par)

types.SimpleNamespace

## 2. <a id='toc2_'></a>[Copy, save and load](#toc0_)

In [9]:
model_copy = model.copy(name='example_copy')
model_copy.save()

In [10]:
model_loaded = MyModelClass(name='example_copy',load=True)
print(model_loaded)

Modelclass: MyModelClass
Name: example_copy

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 Z = ndarray[int32] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sim:
 memory: 0.0 mb



## 3. <a id='toc3_'></a>[To and from dictionary](#toc0_)

In [11]:
modeldict = model.as_dict()
model_dict = MyModelClass(name='example_dict',from_dict=modeldict)
print(model_dict)

Modelclass: MyModelClass
Name: example_dict

namespaces: ['par', 'sol', 'sim']
other_attrs: []
savefolder: saved
cpp_filename: None

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray[float64] with shape = (10,) [0.0 mb]
 Z = ndarray[int32] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sol:
 Y = ndarray[float64] with shape = (10,) [0.0 mb]
 memory: 0.0 mb

sim:
 memory: 0.0 mb



## 4. <a id='toc4_'></a>[Numba](#toc0_)

**Goal:** Call [numba](http://numba.pydata.org/) JIT compilled functions easily using namespaces.

**Problem:** [numba](http://numba.pydata.org/) only allows specific types (and not e.g. `SimpleNamespace`).

**Under-the-hood:** `jit(model)` temporarily turns all namespaces into `namedtuples`, which can be used as input in JIT compilled functions. 

In [12]:
@nb.njit
def fun(par,sol):
    sol.Y[:] = par.X*(par.a+par.b)

In [13]:
def test_numba(model):
    with jit(model) as model_jit:
        fun(model_jit.par,model_jit.sol)

In [14]:
%time test_numba(model)
%time test_numba(model)

CPU times: total: 797 ms
Wall time: 2.15 s
CPU times: total: 0 ns
Wall time: 1e+03 μs


**Note:** The first run is slower due to compiling.

**Check result:**

In [15]:
assert np.allclose(par.X*(par.a+par.b),sol.Y)